## Запускаем базу 

In [1]:
!{ podman stop postgres && podman rm postgres; } || true
!podman run --rm --name postgres -e POSTGRES_PASSWORD=hwpass -e POSTGRES_DB=homework -e POSTGRES_USER=homework -p 5432:5432 -d postgres:16

postgres
Error: no container with ID or name "postgres" found: no such container
366c837477775a226eb708d90973b23b68082e0434633d7c62cfd255109d663c


In [2]:
!podman ps -a

CONTAINER ID  IMAGE                                   COMMAND               CREATED                 STATUS       PORTS                   NAMES
35611b0be42a  docker.io/hyperledger/fabric-ca:latest  bash -c fabric-ca...  6 days ago              Created      0.0.0.0:7054->7054/tcp  ecm-ca.raiffeisen.ru
366c83747777  docker.io/library/postgres:16           postgres              Less than a second ago  Up 1 second  0.0.0.0:5432->5432/tcp  postgres


In [3]:
!{ while ! nc -vzw 3 127.0.0.1 5432; do echo waiting for postgres readiness; done; } || true

localhost [127.0.0.1] 5432 (postgresql) open


In [4]:
!podman logs postgres

The files belonging to this database system will be owned by user "postgres".
This user must also own the server process.

The database cluster will be initialized with locale "en_US.utf8".
The default database encoding has accordingly been set to "UTF8".
The default text search configuration will be set to "english".

Data page checksums are disabled.

fixing permissions on existing directory /var/lib/postgresql/data ... ok
creating subdirectories ... ok
selecting dynamic shared memory implementation ... posix
selecting default max_connections ... 100
selecting default shared_buffers ... 128MB
selecting default time zone ... Etc/UTC
creating configuration files ... ok
running bootstrap script ... ok
performing post-bootstrap initialization ... ok
syncing data to disk ... ok


Success. You can now start the database server using:

    pg_ctl -D /var/lib/postgresql/data -l logfile start

initdb: warning: enabling "trust" authentication for local connections
initdb: hint: You can change 

## Подготавливаем коннект

In [5]:
import pandas as pd
import psycopg2 as pg
import io

/var/folders/4p/7625v4fd3vnfkh0pgq24l678krwr8q/T/ipykernel_45241/3373786476.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
conn = pg.connect(dbname="homework", user="homework", password="hwpass", host="localhost")
cursor = conn.cursor()

## Создаем таблицы

In [7]:
customer_table = """
CREATE TABLE "customer" (
  "customer_id" serial PRIMARY KEY NOT NULL,
  "first_name" varchar(50) NOT NULL,
  "last_name" varchar(50),
  "gender" varchar(30) NOT NULL,
  "DOB" varchar(50),
  "job_title" varchar(50),
  "job_industry_category" varchar(50),
  "wealth_segment" varchar(50) NOT NULL,
  "deceased_indicator" varchar(50) NOT NULL,
  "owns_car" varchar(30) NOT NULL,
  "address" varchar(50) NOT NULL,
  "postcode" varchar(30) NOT NULL,
  "state" varchar(30) NOT NULL,
  "country" varchar(30) NOT NULL,
  "property_valuation" integer NOT NULL
);
"""
cursor.execute(customer_table)
conn.commit()

In [8]:
transaction_table = """
CREATE TABLE "transaction" (
  "transaction_id" serial PRIMARY KEY NOT NULL,
  "product_id" integer NOT NULL,
  "customer_id" integer NOT NULL,
  "transaction_date" varchar(30) NOT NULL,
  "online_order" varchar(30),
  "order_status" varchar(30) NOT NULL,
  "brand" varchar(30),
  "product_line" varchar(30),
  "product_class" varchar(30),
  "product_size" varchar(30),
  "list_price" real NOT NULL,
  "standard_cost" real
);
"""

cursor.execute(transaction_table)
conn.commit()

In [9]:
cursor.execute('ALTER TABLE "transaction" ADD FOREIGN KEY ("customer_id") REFERENCES "customer" ("customer_id");')
conn.commit()

## Заливаем данные

In [10]:
def write_df_to_table(filename: str, table: str, sep: str = ";", null: str = "") -> None:
    with open(filename) as f:
        f.readline() # skip table columns names
        cursor.copy_from(f, table, sep=sep, null=null)
        conn.commit()
    cursor.execute(f"select * from {table} LIMIT 10;")
    conn.commit()
    print(cursor.fetchall())

In [11]:
write_df_to_table("customer.csv", "customer")

[(1, 'Laraine', 'Medendorp', 'F', '1953-10-12', 'Executive Secretary', 'Health', 'Mass Customer', 'N', 'Yes', '060 Morning Avenue', '2016', 'New South Wales', 'Australia', 10), (2, 'Eli', 'Bockman', 'Male', '1980-12-16', 'Administrative Officer', 'Financial Services', 'Mass Customer', 'N', 'Yes', '6 Meadow Vale Court', '2153', 'New South Wales', 'Australia', 10), (3, 'Arlin', 'Dearle', 'Male', '1954-01-20', 'Recruiting Manager', 'Property', 'Mass Customer', 'N', 'Yes', '0 Holy Cross Court', '4211', 'QLD', 'Australia', 9), (4, 'Talbot', None, 'Male', '1961-10-03', None, 'IT', 'Mass Customer', 'N', 'No', '17979 Del Mar Point', '2448', 'New South Wales', 'Australia', 4), (5, 'Sheila-kathryn', 'Calton', 'Female', '1977-05-13', 'Senior Editor', 'n/a', 'Affluent Customer', 'N', 'Yes', '9 Oakridge Court', '3216', 'VIC', 'Australia', 9), (6, 'Curr', 'Duckhouse', 'Male', '1966-09-16', None, 'Retail', 'High Net Worth', 'N', 'Yes', '4 Delaware Trail', '2210', 'New South Wales', 'Australia', 9), (

In [12]:
# Небольшое форматирование чиловых данных
transaction = pd.read_csv("transaction.csv", sep=";", index_col=0)
transaction.list_price = transaction.list_price.map(lambda x: float(x.replace(",", ".")))
transaction.standard_cost = transaction.standard_cost.map(lambda x: x if isinstance(x, float) else float(x.replace(",", ".")))

transaction = transaction[transaction.customer_id <= 4000] # См HW1 ("Убираем транзакции с пользователями, которых не существует в базе)

transaction.to_csv("transaction_fixed.csv", sep=";")
transaction.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19997 entries, 1 to 20000
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product_id        19997 non-null  int64  
 1   customer_id       19997 non-null  int64  
 2   transaction_date  19997 non-null  object 
 3   online_order      19637 non-null  object 
 4   order_status      19997 non-null  object 
 5   brand             19800 non-null  object 
 6   product_line      19800 non-null  object 
 7   product_class     19800 non-null  object 
 8   product_size      19800 non-null  object 
 9   list_price        19997 non-null  float64
 10  standard_cost     19800 non-null  float64
dtypes: float64(2), int64(2), object(7)
memory usage: 1.8+ MB


In [13]:
write_df_to_table("transaction_fixed.csv", "transaction")

[(1, 2, 2950, '25.02.2017', 'False', 'Approved', 'Solex', 'Standard', 'medium', 'medium', 71.49, 53.62), (2, 3, 3120, '21.05.2017', 'True', 'Approved', 'Trek Bicycles', 'Standard', 'medium', 'large', 2091.47, 388.92), (3, 37, 402, '16.10.2017', 'False', 'Approved', 'OHM Cycles', 'Standard', 'low', 'medium', 1793.43, 248.82), (4, 88, 3135, '31.08.2017', 'False', 'Approved', 'Norco Bicycles', 'Standard', 'medium', 'medium', 1198.46, 381.1), (5, 78, 787, '01.10.2017', 'True', 'Approved', 'Giant Bicycles', 'Standard', 'medium', 'large', 1765.3, 709.48), (6, 25, 2339, '08.03.2017', 'True', 'Approved', 'Giant Bicycles', 'Road', 'medium', 'medium', 1538.99, 829.65), (7, 22, 1542, '21.04.2017', 'True', 'Approved', 'WeareA2B', 'Standard', 'medium', 'medium', 60.34, 45.26), (8, 15, 2459, '15.07.2017', 'False', 'Approved', 'WeareA2B', 'Standard', 'medium', 'medium', 1292.84, 13.44), (9, 67, 1305, '10.08.2017', 'False', 'Approved', 'Solex', 'Standard', 'medium', 'large', 1071.23, 380.74), (10, 12,

In [14]:
def execute(statement: str) -> None:
    cursor.execute(statement)
    conn.commit()
    res = cursor.fetchall()
    print(len(res))
    print(res)

## Задание №1 

In [15]:
execute("""
SELECT 
  DISTINCT brand 
FROM 
  transaction 
WHERE 
  standard_cost > 1500
""")

4
[('Solex',), ('Giant Bicycles',), ('OHM Cycles',), ('Trek Bicycles',)]


## Задание №2

In [16]:
execute("""
SELECT 
  * 
FROM 
  transaction 
WHERE 
  order_status = 'Approved' 
  and TO_DATE(transaction_date, 'DD.MM.YYYY') BETWEEN '2017-04-01' 
  AND '2017-04-09';
""")

531
[(17, 79, 2426, '03.04.2017', 'False', 'Approved', 'Norco Bicycles', 'Standard', 'medium', 'medium', 1555.58, 818.01), (19, 54, 2268, '06.04.2017', 'True', 'Approved', 'WeareA2B', 'Standard', 'medium', 'medium', 1292.84, 13.44), (23, 37, 2001, '08.04.2017', 'True', 'Approved', 'OHM Cycles', 'Standard', 'low', 'medium', 1793.43, 248.82), (83, 0, 3398, '01.04.2017', 'True', 'Approved', 'OHM Cycles', 'Standard', 'medium', 'medium', 235.63, 125.07), (89, 0, 2682, '04.04.2017', 'True', 'Approved', 'OHM Cycles', 'Road', 'high', 'large', 12.01, 7.21), (126, 53, 773, '01.04.2017', 'False', 'Approved', 'OHM Cycles', 'Standard', 'medium', 'medium', 795.34, 101.58), (146, 41, 3261, '06.04.2017', 'True', 'Approved', 'Solex', 'Road', 'medium', 'medium', 416.98, 312.74), (154, 3, 3383, '06.04.2017', 'True', 'Approved', 'Trek Bicycles', 'Standard', 'medium', 'large', 2091.47, 388.92), (220, 10, 1502, '09.04.2017', 'True', 'Approved', 'WeareA2B', 'Touring', 'medium', 'medium', 1466.68, 363.25), (2

## Задание №3

In [17]:
execute("""
SELECT 
  DISTINCT job_title 
FROM 
  customer 
WHERE 
  (
    job_industry_category = 'IT' 
    OR job_industry_category = 'Financial Services'
  ) 
  AND job_title :: varchar(10) like 'Senior%'
""")

6
[('Senior Cost Accountant',), ('Senior Developer',), ('Senior Editor',), ('Senior Financial Analyst',), ('Senior Quality Engineer',), ('Senior Sales Associate',)]


## Задание №4

In [18]:
execute("""
SELECT 
  DISTINCT brand 
FROM 
  transaction 
WHERE 
  customer_id IN (
    SELECT 
      customer_id 
    FROM 
      customer 
    WHERE 
      job_industry_category = 'Financial Services'
  )
""")

7
[('Giant Bicycles',), ('Norco Bicycles',), ('OHM Cycles',), ('Solex',), ('Trek Bicycles',), ('WeareA2B',), (None,)]


## Задание №5

In [19]:
execute("""
SELECT 
  * 
FROM 
  customer 
WHERE 
  customer_id IN (
    SELECT 
      customer_id 
    FROM 
      transaction 
    WHERE 
      brand = 'Giant Bicycles' 
      OR brand = 'Norco Bicycles' 
      OR brand = 'Trek Bicycles'
  ) 
LIMIT 
  10
""")

10
[(1, 'Laraine', 'Medendorp', 'F', '1953-10-12', 'Executive Secretary', 'Health', 'Mass Customer', 'N', 'Yes', '060 Morning Avenue', '2016', 'New South Wales', 'Australia', 10), (2, 'Eli', 'Bockman', 'Male', '1980-12-16', 'Administrative Officer', 'Financial Services', 'Mass Customer', 'N', 'Yes', '6 Meadow Vale Court', '2153', 'New South Wales', 'Australia', 10), (3, 'Arlin', 'Dearle', 'Male', '1954-01-20', 'Recruiting Manager', 'Property', 'Mass Customer', 'N', 'Yes', '0 Holy Cross Court', '4211', 'QLD', 'Australia', 9), (4, 'Talbot', None, 'Male', '1961-10-03', None, 'IT', 'Mass Customer', 'N', 'No', '17979 Del Mar Point', '2448', 'New South Wales', 'Australia', 4), (5, 'Sheila-kathryn', 'Calton', 'Female', '1977-05-13', 'Senior Editor', 'n/a', 'Affluent Customer', 'N', 'Yes', '9 Oakridge Court', '3216', 'VIC', 'Australia', 9), (6, 'Curr', 'Duckhouse', 'Male', '1966-09-16', None, 'Retail', 'High Net Worth', 'N', 'Yes', '4 Delaware Trail', '2210', 'New South Wales', 'Australia', 9)

## Задание №6

In [20]:
execute("""
SELECT 
  * 
FROM 
  customer 
WHERE 
  customer_id NOT IN (
    SELECT 
      customer_id 
    FROM 
      transaction
  )
""")

507
[(852, 'Andie', 'Bonney', 'Female', '2000-11-04', 'Compensation Analyst', 'Financial Services', 'Affluent Customer', 'N', 'Yes', '94 Anhalt Way', '3139', 'VIC', 'Australia', 7), (869, 'Addia', 'Abels', 'Female', '1984-03-11', 'Account Representative I', 'Financial Services', 'High Net Worth', 'N', 'Yes', '02377 Maywood Trail', '2287', 'NSW', 'Australia', 7), (1373, 'Shaylynn', 'Epsley', 'Female', '1958-09-23', 'Director of Sales', 'Financial Services', 'Mass Customer', 'N', 'Yes', '0 Grasskamp Pass', '3170', 'VIC', 'Australia', 10), (2074, 'Roslyn', 'Rawdall', 'Female', '1997-06-11', None, 'Financial Services', 'Mass Customer', 'N', 'No', '95483 Washington Junction', '2505', 'NSW', 'Australia', 9), (2660, 'Hunt', 'Scollard', 'Male', '1963-11-15', None, 'Retail', 'High Net Worth', 'N', 'Yes', '359 Briar Crest Road', '2155', 'NSW', 'Australia', 10), (3229, 'Carrol', 'Gheorghie', 'Male', '1964-10-18', 'Sales Representative', 'Retail', 'High Net Worth', 'N', 'Yes', '44 Service Alley', 

## Задание №7

In [21]:
execute("""
SELECT 
  * 
FROM 
  customer 
WHERE 
  job_industry_category = 'IT' 
  AND customer_id IN (
    SELECT 
      customer_id 
    FROM 
      transaction 
    WHERE 
      standard_cost = (
        SELECT 
          MAX(standard_cost) 
        FROM 
          transaction
      )
  )
""")

9
[(34, 'Jephthah', 'Bachmann', 'U', '1843-12-21', 'Legal Assistant', 'IT', 'Affluent Customer', 'N', 'No', '90 Lawn Parkway', '4805', 'QLD', 'Australia', 4), (893, 'Gibby', 'Fearnley', 'Male', '1983-09-11', 'Geologist I', 'IT', 'Mass Customer', 'N', 'No', '6382 Bayside Street', '2153', 'NSW', 'Australia', 10), (975, 'Goldarina', 'Rzehorz', 'U', None, 'Automation Specialist IV', 'IT', 'Mass Customer', 'N', 'No', '938 Monica Park', '3173', 'VIC', 'Australia', 9), (1672, 'Sharla', 'Creebo', 'Female', '1963-04-27', 'Design Engineer', 'IT', 'Affluent Customer', 'N', 'Yes', '1 Morning Circle', '2250', 'NSW', 'Australia', 8), (1773, 'Nickolas', 'Guittet', 'U', None, None, 'IT', 'Mass Customer', 'N', 'Yes', '835 West Hill', '2118', 'NSW', 'Australia', 11), (1918, 'Devin', 'Sandeson', 'U', None, 'Staff Accountant II', 'IT', 'Affluent Customer', 'N', 'No', '51763 Carey Place', '3056', 'VIC', 'Australia', 7), (2913, 'Padraic', 'Bonnar', 'Male', '1955-07-11', 'VP Quality Control', 'IT', 'Affluent

## Задание №8

In [22]:
execute("""
SELECT 
  * 
FROM 
  customer 
WHERE 
  (
    job_industry_category = 'IT' 
    OR job_industry_category = 'Health'
  ) 
  AND customer_id IN (
    SELECT 
      customer_id 
    FROM 
      transaction 
    WHERE 
      order_status = 'Approved' 
      AND TO_DATE(transaction_date, 'DD.MM.YYYY') BETWEEN '2017-07-07' 
      AND '2017-07-17'
  )
""")

115
[(1489, 'Mead', 'Wollard', 'Male', '1993-05-13', 'Teacher', 'Health', 'Mass Customer', 'N', 'No', '9 Sycamore Pass', '2287', 'NSW', 'Australia', 5), (753, 'Josy', 'St. Quentin', 'Female', '1970-03-07', 'Food Chemist', 'Health', 'Affluent Customer', 'Y', 'Yes', '195 Knutson Place', '4019', 'QLD', 'Australia', 8), (1091, 'Keith', 'Housden', 'Male', '1980-04-15', None, 'Health', 'Mass Customer', 'N', 'No', '04 Onsgard Center', '2101', 'NSW', 'Australia', 9), (2285, 'Maritsa', 'Bellison', 'Female', '1973-10-27', 'Quality Control Specialist', 'Health', 'High Net Worth', 'N', 'No', '09356 Badeau Drive', '3630', 'VIC', 'Australia', 1), (350, 'Carmela', 'Jesper', 'Female', '1977-12-09', 'Clinical Specialist', 'Health', 'Mass Customer', 'N', 'Yes', '0293 Porter Crossing', '2830', 'NSW', 'Australia', 5), (424, 'Dennie', 'Eunson', 'Male', '1978-01-25', 'Research Nurse', 'Health', 'High Net Worth', 'N', 'No', '8 Moland Pass', '2065', 'NSW', 'Australia', 10), (1683, 'Brenn', 'Bacon', 'U', None,

## Закрываем коннекты

In [23]:
cursor.close()
conn.close()